# dag_prf_utils demos

A basic overview on surface plotting, and colormaps: 

[1] Freeview based (I almost exclusively use this)

[2] Blender (maybe it will be useful?)

[3] Generic .ply format (maybe it will be useful?)

[4] Colormaps

*Note that for many cases you will be better off using pycortex; this simply provides a couple of alternative methods, should you want to do that.*

In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import os


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
sub = 'sub-01'                              # Subject number
fs_dir = os.envi                                    # e.g., /home/project1/derivatives/freesurfer

# Load some data to plot on the surface (e.g., PRF eccentricity and polar angle):
# -> should 1D np.ndarray, where the length = number of vertices in subject surface

polar_angle_data = np.load('path/to/subjects/polar/angle/data')
eccentricity_data = np.load('path/to/subjects/polar/angle/data')

# ** Control visibility of data ** [using mask, or alpha, or both...] 
# If you don't want to show the values of every point (for example because it is outside the visual cortex). You may want to hide it. 
# If you are plotting PRFs, then you may want to hide the bad fits. So you can create a mask for where the rsq<threshold (e.g., 0.1)
# data_mask: what to show (TRUE), what to hide (FALSE)
# -> should boolean 1D np.ndarray, where the length = number of vertices in subject surface
# -> if unspecified, all surface functions assume TRUE for all voxels
data_mask = np.load('path/to/some/mask')

# data_alpha: transparency (invisible = 0), (opaque = 1).
# -> should be 1D np.ndarray, min=0, max=1.  where the length = number of vertices in subject surface
# -> you could for example take the rsquared values of PRF fits and use this to determine the visibility of the prf data...
data_alpha = np.load('path/to/some/alpha/values')


### [1] Using freeview
* requires freeview 
* Specify the data, the mesh, and the mask (you can only use binary masking, the option for varying the transparency is not available here). 
* scripts will create a custom surf file, and the command (which contains the colormap info) to open it in freeview
* The colormap can be anything from matplotlib. Just specify the min and max values. (https://matplotlib.org/stable/tutorials/colors/colormaps.html)
* You can also specify the camera angle for when freeview opens, and ask it to automatically take a picture of the surface. This can be useful if you want to iterate through several subjects/surface plots and save the figures as pngs, but can't be bothered to sit and click again and again... 

In [ ]:
from dag_prf_utils.fs_tools import FSMaker
fs = FSMaker(sub=sub,fs_dir=fs_dir)

# Add polar angle plot
fs.add_surface(
    data = polar_angle_data,
    surf_name = f'{sub}-polar_angle',    
    vmin = -3.14, vmax=3.14, # min and max values of polar anlge 
    data_mask=data_mask,
    cmap = 'pol', # I have a custom color map that I got from a colleague for polar angle. (can also use 'hsv' instead)
)

# Add eccentricity
fs.add_surface(
    data = eccentricity_data,
    surf_name = f'{sub}-eccentricity',    
    vmin = 0, vmax = 5, # min and max values of eccentricity
    data_mask=data_mask,
    cmap = 'ecc', # Also a custom map for eccentricity, which scales nicely. (again you can use anything you like, including developing your own. ) 
)

In [ ]:
# Now we can open one of the surfaces in freeview
fs.open_fs_surface(
    surf_name=f'{sub}-polar_angle',
    mesh = 'inflated',          # what type of surface? inflated? pial?
    )

In [ ]:
# Maybe we want to open with a specific cameram angle and take a screenshot?
fs.open_fs_surface(
    surf_name=f'{sub}-polar_angle',
    mesh = 'inflated',          # what type of surface? inflated? pial?
    do_scrn_shot = True,
    scr_shot_file = 'where/to/put/the/screenshot', # Optional...Can just put it in custom surf folder
    # *** camer angles ***
    azimuth = 10, zoom = 1, elevation=5, roll=0, 
    )

***

### Blender
* requires freesurfer and blender
* This is the most powerful approach and allows for a lot of customization (due to the blender api flexibility, which can be called via python)
* The script will load the inflated and pial mesh, with the option to slide between the 2 (i.e. customize how inflated you want the surface to be)
* You can load several colormaps at once, and flip between them 
* If you are feeling adventurous, you can even create an animation over time, (e.g., plot timecourse info on the surface). This is a bit experimental, and may take up a lot of data and computing power. I haven't fully explored it. 


In [ ]:
from dag_prf_utils.blender_tools import BlendMaker
bm = BlendMaker(
    sub=sub,
    fs_dir=fs_dir,
    out_dir='where/to/save/the/blender/files',    
)
# This will create a folder with 4 .ply meshes
# -> left-inflated, right-inflated, left-pial, right-pial
# & 2 csv files with rgb information for:
# -> cortical curvature, cortical thickness

In [ ]:

# Add polar angle plot
bm.add_cmap(
    data=polar_angle_data,
    surf_name=f'{sub}-polar_angle',
    data_mask=data_mask,
    data_alpha=data_alpha,
    vmin = -3.14, vmax=3.14, # min and max values of polar anlge 
    cmap = 'pol',
)
# Add eccentricity
bm.add_cmap(
    data=eccentricity_data,
    surf_name=f'{sub}-ecc',
    data_mask=data_mask,
    data_alpha=data_alpha,
    vmin = 0, vmax=5, # min and max values of polar anlge 
    cmap = 'ecc',
)

# This will add 2 more csv files with rgb information for:
# -> polar_angle, and eccentricity

In [ ]:
# Now we can launch blender, and load in the .ply files and the rgb data...
bm.launch_blender()


This may take a while. But once it is done, you can save everything together as one .blender file. You can open this file on its own, and everything will load very quickly. (so you only need to run this once, unless you change the data...)

You can then do a number of cool things:

* Inflate & deflate the hemispheres:    
    * Select the hemisphere (click on it; it should then have an orange line outlining it). 
    * Click on the green triangle on the panel on the right
    * Got to "shape keys"
    * Click on interpolated
    * There is a slider named "Value". Change the value here from 0-1 to inflate & deflate the hemisphere
* Switch between different plots:
    * Select the hemisphere (click on it)
    * Click on the green triangle on the panel on the right
    * Go to color attributes
    * Click on what you want to show on the surface (e.g., polar_angle, curvature etc.)

***

### Generic .ply format
* requires freesurfer 
* Specify the data, the mesh, and the mask (including an option for variable transparency). 
* can create a single .ply file (per hemisphere), which contains all the information about the mesh (vx coordinates, face id); the data values for each vertex, and a color value for each vertex (determined by the data, and specified colormap). 
* This can be opened by most 3D viewing software (e.g., meshlab, blender)...


In [ ]:
from dag_prf_utils.mesh_maker import dag_fs_to_ply

dag_fs_to_ply(
    sub=sub, 
    data=polar_angle_data, 
    fs_dir=fs_dir, 
    mesh_name='inflated',                   # Could be pial
    out_dir='where/to/save/the/ply/files',  # Output (multiple .ply files)
    under_surf='curv',                      # What is going underneath the data (e.g., curvature)
    # *** OPTIONAL ***
    data_mask = data_mask,
    data_alpha = data_alpha, 
    surf_name = f'{sub}-polar_angle',
    cmap = 'pol',
    vmin = -3.14, vmax=3.14,
    )
# Now you can just navigate to out dir, and view the surface (.ply) file using any 3D renderer (e.g., meshlab, blender)

# ... same again, but for eccentricity...
dag_fs_to_ply(
    sub=sub, 
    data=eccentricity_data, 
    fs_dir=fs_dir, 
    mesh_name='inflated',                   # Could be pial
    out_dir='where/to/save/the/ply/files',  # Output (multiple .ply files)
    under_surf='curv',                      # What is going underneath the data (e.g., curvature)
    # *** OPTIONAL ***
    data_mask = data_mask,
    data_alpha = data_alpha, 
    surf_name = f'{sub}-eccentricity',
    cmap = 'jet',
    vmin = 0, vmax=5,
    )

***

# A quick note on colormaps
You can use any colormap specified in (https://matplotlib.org/stable/tutorials/colors/colormaps.html)

I also have a function which allows you to use a color picker to work out a nice custom colormap, and then save it. 
In bash try running:
> dag_cmap_maker --vmin -3.14 --vmax 3.14 --n_steps 9 


You will see a plot with a random colormap. You can click on any of the points in the rgb graph, and move them around. When you are happy, close the window, and you will see the option to save the colormap in the bash terminal. This will put everything into a .json (cmaps.json). This toolbox will be able to reconstruct this anytime you want. (using dag_get_cmap)

This example could help you make a custom polar angle map. 

Also see below how you can create your own custom colormap from lists of colors.

In [ ]:
from dag_prf_utils.plot_functions import *

demo_mat = np.ones((10,100)) * np.arange(100)
# You can load any of the defualt colormaps that exist in matplotlib
eg_default_cmaps = {}
eg_default_cmaps['jet'] = dag_get_cmap('jet')
eg_default_cmaps['jet_r'] = dag_get_cmap('jet_r')
eg_default_cmaps['hsv'] = dag_get_cmap('hsv')

for eg_cmap in ['jet', 'jet_r', 'hsv']:
    plt.figure()
    plt.imshow(demo_mat, cmap=eg_default_cmaps[eg_cmap])
    plt.title(eg_cmap)


In [ ]:
# To create a custom color map:
# [1] cmap_name             name of your new cmap
# [2] col_list              list of colors (any length, just doing 3 here to keep it simple)
# [3] (optional) col_steps  list of numbers (how to space the list of colors...)

cmap_name = 'custom_cmap1'
col_list = [
    (255,0,0),
    (0,255,0),
    (0, 0, 0)
]

# Put this together in a dictionary:
cmap1 = dag_get_cmap(
    cmap_name=cmap_name,
    col_list=col_list,
)
plt.figure()
plt.imshow(demo_mat, cmap=cmap1)
plt.title('custom 1')


# You can also specify the spacing of the colors using a third component: col_steps
cmap2 = dag_get_cmap(
    cmap_name=cmap_name,
    col_list=col_list,
    col_steps=[0,.1,2]    
)
plt.figure()
plt.imshow(demo_mat, cmap=cmap2)
plt.title('Squish and stretch the cmap using col_steps')


# Nonlinear spacing
cmap3 = dag_get_cmap(
    cmap_name=cmap_name,
    col_list=col_list,
    col_steps=[0,1.9,2]    
)
plt.figure()
plt.imshow(demo_mat, cmap=cmap3)
plt.title('Squish and stretch the cmap using col_steps')

In [ ]:
# This also supports a variety of ways of defining colors:
cmap = dag_get_cmap(
    cmap_name='blah',
    col_list = ['r', 'b', 'c'],
)
plt.figure()
plt.imshow(demo_mat, cmap=cmap)
plt.title('Using matplotlib color letters')


cmap = dag_get_cmap(
    cmap_name='blah',
    col_list = ['red', 'cyan', 'green'],)
plt.figure()
plt.imshow(demo_mat, cmap=cmap)
plt.title('Using matplotlib color names')

cmap = dag_get_cmap(
    cmap_name='blah',
    col_list = ['#1b9e77','#d95f02','#7570b3'])
plt.figure()
plt.imshow(demo_mat, cmap=cmap)
plt.title('Using HEX colors')

